In [2]:
import sys
import os
sys.path.append('..')

In [3]:
from src.dataset import ImageDataLoader
from src import config 
from src.utils import get_balanced_dataset_sampler, get_train_transform, get_test_transform
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, Subset
import torch
from src.metrics import Metrics
from pathlib import Path

In [ ]:
train_data_size = []
test_data_size = []
skfold = StratifiedKFold(n_splits=5, shuffle=True)
data_loader = ImageDataLoader('../data/binary/hiper_others/')
binary_labels = [sample[1] for sample in data_loader.dataset.samples]
for fold, (train_ids, test_ids) in enumerate(skfold.split(data_loader.dataset, binary_labels)):

    train_subset = Subset(data_loader.dataset, train_ids)
    train_subset.transform = get_train_transform()
    sampler = get_balanced_dataset_sampler(data_loader, train_ids, train_subset)
    train_loader = DataLoader(train_subset, batch_size=config.BATCH_SIZE, sampler=sampler, num_workers=config.NUM_WORKERS)

    test_subset = Subset(data_loader.dataset, test_ids) 
    test_subset.transform = get_test_transform()
    val_loader = DataLoader(test_subset, batch_size=config.BATCH_SIZE, num_workers=config.NUM_WORKERS, shuffle=True )
    num_target = 0
    num_others = 0
    
    for epoch in range(1):
        for data, labels in train_loader:
            num_target += torch.sum(labels == 0)
            num_others += torch.sum(labels == 1)

    train_data_size.extend([num_target.item(), num_others.item()])
    print(num_target.item())
    print(num_others.item())
    
    num_target = 0
    num_others = 0
    for epoch in range(1):
        for data, labels in val_loader:
            num_target += torch.sum(labels == 0)
            num_others += torch.sum(labels == 1)

    test_data_size.extend([num_target.item(), num_others.item()])
    print(num_target.item())
    print(num_others.item())
    break



In [ ]:
import matplotlib.pyplot as plt 

fig = plt.figure(figsize = (4, 3))
# creating the bar plot
plt.bar(labels, values,
        width = 0.4, )

plt.xlabel("Class")
plt.ylabel("quantity")
plt.title("Data set distribution")
for index, value in enumerate(values):
    plt.text(index,value+0.01, str(value))
plt.show()

In [ ]:
metrics_result = Metrics()
metrics_result.compute_metrics([1,0,1,0], [1,0,1,1])
metrics_result.accuracy

In [5]:
def get_subfolder_paths(directory):
    directory = Path(directory)
    subfolder_paths = []
    
    for path in directory.rglob('*'):
        if path.is_dir():
            subfolder_paths.append(path)
    
    return subfolder_paths

# Example usage
directory_path = '../data/binary/podoc_others/'
subfolders = get_subfolder_paths(directory_path)

# Print the subfolder paths
for subfolder in subfolders:
    print(subfolder)

..\data\binary\podoc_others\others
..\data\binary\podoc_others\_podocitopatia
..\data\binary\podoc_others\others\Crescentes
..\data\binary\podoc_others\others\hipercellularity
..\data\binary\podoc_others\others\membranous
..\data\binary\podoc_others\others\normal
..\data\binary\podoc_others\others\sclerosis
..\data\binary\podoc_others\others\Crescentes\AZAN
..\data\binary\podoc_others\others\Crescentes\H_E
..\data\binary\podoc_others\others\Crescentes\PAMS
..\data\binary\podoc_others\others\Crescentes\PAS
..\data\binary\podoc_others\others\Crescentes\PICRO
..\data\binary\podoc_others\others\Crescentes\PS
..\data\binary\podoc_others\others\hipercellularity\AZAN
..\data\binary\podoc_others\others\hipercellularity\HE
..\data\binary\podoc_others\others\hipercellularity\PAS
..\data\binary\podoc_others\others\hipercellularity\PS
..\data\binary\podoc_others\others\hipercellularity\PSA
..\data\binary\podoc_others\others\membranous\primary_membranous
..\data\binary\podoc_others\others\membranou